<a href="https://colab.research.google.com/github/tejask666/Tejas_INFO5731_Spring2020/blob/master/Mayuri%20Final%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
import os
import re,datetime
import time
import csv
from selenium.common.exceptions import NoSuchElementException
from google.colab import files
from selenium import webdriver
  
class In_Class_Assigment():
    def get_data(self,list_of_journal):
        list_of_data=[]
        print(list_of_journal)
        # Launch chrome browser.
        obj = In_Class_Assigment()
        options = webdriver.ChromeOptions()
        # Runs browser in background.
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument("--window-size=1920,1080")
        options.add_argument("start-maximized")
        options.add_argument("disable-infobars")  
        options.add_argument("--disable-extensions")
        driver = webdriver.Chrome('chromedriver',options=options)
        driver.maximize_window()
        #This loop will run twice as there are two journals.
        for journal in range(len(list_of_journal)):
            with open(r'/content/Book1.csv', 'a', newline='') as file:
                writer = csv.writer(file)
                writer.writerow([list_of_journal[journal][3]])
            list_of_data = list_of_journal[journal]
         #driver.get is use to access url.
            driver.get(list_of_data[2])
            driver.find_element_by_xpath("//input[@name='user']").send_keys("tpk0023")
            driver.find_element_by_xpath("//input[@name='pass']").send_keys("Jan@022019")
          # finds year,but before that it checks it is already expanded or not.   
            driver.find_element_by_xpath("//input[@type='submit']").click()
            time.sleep(2)
            expand_years = driver.find_elements_by_xpath("//i[@class='icon-add_box']//following::span[1]")
            for expand in range(len(expand_years)):  # count
                print("*************in expand years*************")
                expand_years = driver.find_elements_by_xpath("//i[@class='icon-add_box']//following::span[1]")
                expand_years[expand].click()
                year_range = expand_years[expand].text
                y = year_range.split(" - ")
                year_range = y[0] + "-" + y[1]
                sublist= "//ul[@id='#value']//li"
                sublist=sublist.replace('#value',year_range)
                print(sublist)
                sub_list=driver.find_elements_by_xpath(sublist)
                for i in range(len(sub_list)):
                    print("***************in sublist***********")
                    sub_list = driver.find_elements_by_xpath(sublist)
                    sub_list[i].click()
                    time.sleep(5)
                    issue_list= driver.find_elements_by_xpath("(//a[contains(text(),'Volume')])")
                    for issue in range(len(issue_list)):#
                        print("************in issue list**********")
             #This finds webelement i.e.location of references  
                        issue_list = driver.find_elements_by_xpath("(//a[contains(text(),'Volume')])")
                        issue_list[issue].click()
                        time.sleep(18)
                        ref_list=driver.find_elements_by_xpath("//a[@title='References']")
             # This loop will run for the number pages i.e. on which articles are available.            
                        for ref in range(len(ref_list)):#
                            time.sleep(30)
                            print("************in reflist****************")
                            ref_list = driver.find_elements_by_xpath("//a[@title='References']")
                            ref_list[ref].click()
                            time.sleep(15)
                            try:
                                ref_tab=driver.find_elements_by_xpath("(//span[text()='REFERENCES'])[2]")
                                if len(ref_tab)>0:
              #driver.execute_script("arguments[0].scrollIntoView();",ref_tab)
                                    ref_tab[0].click()
                                    actual_ref = driver.find_elements_by_xpath("//div[@class='accordion__content']//ul//li")
                                    if actual_ref != []:
                                        for r in range (1,len(actual_ref)):#1,
                                            print("in references tab")
                                            actual_ref = driver.find_elements_by_xpath("//div[@class='accordion__content']//ul//li")
                                            if len(actual_ref) !=[]:
                                               reference_text= actual_ref[r].text
             # Actual content i.e. data which we want to extract from webpage is saved in reference_text in 
                                               author=obj.get_author(reference_text)
                                               if author != None:
                                                  author.append(list_of_data[3])
                                                  obj.writeintocsv(author)
                                               else:
                                                  continue
                                    #driver.back()
                                        print("returned")
                                        driver.back()
                                else:
                                  driver.back()
                                  continue        
                            except NoSuchElementException:
                                citing=driver.find_elements_by_xpath("//span[text()='Citing Literature']")
                                if len(citing)>0:
                                  citing[0].click()
                                  time.sleep(3)
                                  actual_ref = driver.find_elements_by_xpath("//li[@class='citedByEntry']")
                                  if actual_ref != []:
                                    for r in range(1,len(actual_ref)):#
                                        print("******************in citing literatur*************")
                                        actual_ref = driver.find_elements_by_xpath("//li[@class='citedByEntry']")
                                        reference_text = actual_ref[r].text
                                        author = obj.get_author(reference_text)
                                        if author != None:
                                           author.append(list_of_data[3])
                                           obj.writeintocsv(author)
                                        else:
                                          continue
                                    #driver.back()    #    driver.execute_script("arguments[0].scrollIntoView();", list_of_articles[i])
                                    driver.back()
                                else:
                                  driver.back()
                                  continue    
                            except:
                              driver.back()
                              continue
                                    
                            #    driver.execute_script("arguments[0].scrollIntoView();", list_of_articles[i])
                        driver.back()
                driver.back()
            driver.back()
            # button = driver.find_element_by_xpath("(//img[contains(@src,'/images/arrow-right.png')])[1]")
            # driver.execute_script("arguments[0].scrollIntoView();", button)
            # button.click()
            # time.sleep(10)

    def get_author(self, reference):
        author_detail = []
        tapp = []
        tapp[:0] = reference
         #Separates author name and year, as year is mentioned in () with sentences and extracting year. 
        remove = "1234567890"
        temp = ""
        for i in range(len(tapp)):
            if tapp[i] in remove:
                break
            else:
                temp = temp + tapp[i]
        author_name = temp.strip(".(")
        try:
          # Finding 4 digit in reference 
            match = re.search(r'\d{4}', reference)
          # Extracting year  
            date = datetime.datetime.strptime(match.group(), '%Y').date()
            year = date.year
        except:
            author_detail = None
            return author_detail
           # Spliting article and year as year is always after dot  
        article_title = reference.split(").")
        if len(article_title) > 2:
            try:
                title = re.search("(?P<url>https?://[^\s]+)", article_title[2]).group("url")
            except:
                list_of_sentence = article_title[2].split(".")
                title = list_of_sentence[0].strip(" ")
        else:
            if len(article_title) == 1:
                try:
                    title = re.search("(?P<url>https?://[^\s]+)", article_title[2]).group("url")
                except:
                    title = article_title[0]
            else:
                list_of_sen = article_title[1].split(".")
                title = list_of_sen[0].strip(" ")
         #checking if author or year or author name any of the detail is not available it will none the author details        
        if title == "" or year == "" or author_name == "":
            author_detail = None
            return author_detail
        else:
            author_detail.append(title)
            author_detail.append(year)
            author_detail.append(author_name)
            return author_detail
    #writing all information in csv file        

    def writeintocsv(self, ref_details):
        print(ref_details)
        with open(r'/content/Book1.csv', 'a', encoding="utf-8") as file:
            writer = csv.writer(file)
            if len(ref_details) <= 2:
                pass
            else:
                writer.writerow(["", ref_details[0], ref_details[1], str(ref_details[2])])
    # This checks file already exists or not on path
object = In_Class_Assigment()
isExist = os.path.exists(r'/content/Book1.csv')
if isExist:
    os.remove(r'/content/Book1.csv')
    #  if file already exists it will append in that otherwise it will create new file and append data in it.
with open(r'/content/Book1.csv', 'a+', newline='') as file:
   # Object created to use csv.writer applications. 
    writer = csv.writer(file)
    writer.writerow(["Journal Names", "Article Title", "Year", "Author"])
data_list=[("1968", "2019","https://libproxy.library.unt.edu:4060/loi/19383703/year/1982","Applied Behavior Analysis"),("1958", "2019","https://libproxy.library.unt.edu:4060/loi/19383711/year/","Experimental Analysis of Behavior")]
object.get_data(data_list)